# Descripción del proyecto
La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.
Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.
Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.
Instrucciones del proyecto.
## 	Abre y examina el archivo de datos. Dirección al archivo:datasets/users_behavior.csv Descarga el dataset
## 	Segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.
## 	Investiga la calidad de diferentes modelos cambiando los hiperparámetros. Describe brevemente los hallazgos del estudio.
## 	Comprueba la calidad del modelo usando el conjunto de prueba.
## 	Tarea adicional: haz una prueba de cordura al modelo. Estos datos son más complejos que los que habías usado antes así que no será una tarea fácil. Más adelante lo veremos con más detalle.
# Descripción de datos
Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. La información dada es la siguiente:
•	сalls — número de llamadas,
•	minutes — duración total de la llamada en minutos,
•	messages — número de mensajes de texto,
•	mb_used — Tráfico de Internet utilizado en MB,
•	is_ultra — plan para el mes actual (Ultra - 1, Smart - 0).

# Paso 1: Cargar y examinar datos

In [1]:
import pandas as pd

In [2]:
# Cargar los datos desde el archivo CSV
data = pd.read_csv('/datasets/users_behavior.csv')

# Mostrar las primeras filas del dataset y la información general
data.head(), data.info(), data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


(   calls  minutes  messages   mb_used  is_ultra
 0   40.0   311.90      83.0  19915.42         0
 1   85.0   516.75      56.0  22696.96         0
 2   77.0   467.66      86.0  21060.45         0
 3  106.0   745.53      81.0   8437.39         1
 4   66.0   418.74       1.0  14502.75         0,
 None,
              calls      minutes     messages       mb_used     is_ultra
 count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
 mean     63.038892   438.208787    38.281269  17207.673836     0.306472
 std      33.236368   234.569872    36.148326   7570.968246     0.461100
 min       0.000000     0.000000     0.000000      0.000000     0.000000
 25%      40.000000   274.575000     9.000000  12491.902500     0.000000
 50%      62.000000   430.600000    30.000000  16943.235000     0.000000
 75%      82.000000   571.927500    57.000000  21424.700000     1.000000
 max     244.000000  1632.060000   224.000000  49745.730000     1.000000)

# Paso 2: Segmentación de los Datos

In [3]:
from sklearn.model_selection import train_test_split

# Primero, dividimos los datos en entrenamiento y temporal (combinando validación y prueba)
train_data, temp_data = train_test_split(data, test_size=0.4, random_state=123)

# Luego, dividimos los datos temporales en validación y prueba
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=123)

# Imprimir las dimensiones de cada conjunto
print("Tamaño del conjunto de entrenamiento:", train_data.shape)
print("Tamaño del conjunto de validación:", validation_data.shape)
print("Tamaño del conjunto de prueba:", test_data.shape)

Tamaño del conjunto de entrenamiento: (1928, 5)
Tamaño del conjunto de validación: (643, 5)
Tamaño del conjunto de prueba: (643, 5)


# Paso 3: Investigación de Modelos

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Preparar los conjuntos de datos
features_train = train_data.drop(columns=['is_ultra'])
target_train = train_data['is_ultra']
features_validation = validation_data.drop(columns=['is_ultra'])
target_validation = validation_data['is_ultra']

# Modelos a evaluar
models = {
    "Logistic Regression": LogisticRegression(random_state=123, max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=123),
    "Random Forest": RandomForestClassifier(random_state=123)
}

# Entrenar y evaluar cada modelo
results = {}
for name, model in models.items():
    model.fit(features_train, target_train)
    predictions = model.predict(features_validation)
    accuracy = accuracy_score(target_validation, predictions)
    results[name] = accuracy

results

{'Logistic Regression': 0.7589424572317263,
 'Decision Tree': 0.7465007776049767,
 'Random Forest': 0.8180404354587869}

# Paso 4: Ajuste de Hiperparámetros

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Definir la cuadrícula de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None]
}

# Inicializar el modelo GridSearchCV con el modelo RandomForest y la cuadrícula de parámetros
grid_search = GridSearchCV(RandomForestClassifier(random_state=123), param_grid, cv=3, scoring='accuracy')

# Entrenar GridSearchCV en los datos de entrenamiento
grid_search.fit(features_train, target_train)

# Obtener el mejor modelo y su puntuación
best_model = grid_search.best_estimator_
best_accuracy = grid_search.best_score_

# Imprimir los mejores hiperparámetros y la mejor exactitud
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor exactitud de cross-validation:", best_accuracy)

Mejores hiperparámetros: {'max_depth': 10, 'n_estimators': 100}
Mejor exactitud de cross-validation: 0.8034185549628639


# Paso 5: Evaluación Final en el Conjunto de Prueba

In [6]:
from sklearn.metrics import accuracy_score

# Preparar las características y la variable objetivo del conjunto de prueba
features_test = test_data.drop(columns=['is_ultra'])
target_test = test_data['is_ultra']

# Utilizar el mejor modelo para hacer predicciones en el conjunto de prueba
test_predictions = best_model.predict(features_test)

# Calcular y mostrar la exactitud en el conjunto de prueba
test_accuracy = accuracy_score(target_test, test_predictions)
print("Exactitud en el conjunto de prueba:", test_accuracy)

Exactitud en el conjunto de prueba: 0.7931570762052877


# Prueba de Cordura (Paso 5 Adicional)

In [7]:
import numpy as np

# Crear datos de entrada extremos
extreme_data = pd.DataFrame({
    'calls': [0, 300],  # Muy pocas llamadas y muchas llamadas
    'minutes': [0, 2000],  # Ninguna duración y duración muy larga
    'messages': [0, 500],  # Ningún mensaje y muchos mensajes
    'mb_used': [0, 50000]  # Sin uso de datos y uso extremadamente alto
})

# Hacer predicciones con el mejor modelo
extreme_predictions = best_model.predict(extreme_data)
print("Predicciones para datos extremos:", extreme_predictions)

Predicciones para datos extremos: [1 1]


In [8]:
importances = best_model.feature_importances_
features = ['calls', 'minutes', 'messages', 'mb_used']
importance_dict = dict(zip(features, importances))

print("Importancia de las características:", importance_dict)

Importancia de las características: {'calls': 0.21865436174556474, 'minutes': 0.22983324249122952, 'messages': 0.21606859535819978, 'mb_used': 0.3354438004050059}


Predicciones para Datos Extremos:

El modelo predice "1" (plan Ultra) para ambos casos extremos (mínimos y máximos en uso de llamadas, minutos, mensajes y datos). Esto sugiere que el modelo podría estar sesgado hacia la clasificación de usuarios en el plan Ultra cuando se presentan valores extremos. Este comportamiento puede ser un área de interés para investigar más a fondo, especialmente en términos de cómo el balance de clases y la distribución de las características afectan el entrenamiento del modelo.

Importancia de las Características:

mb_used: 33.54% - Esta es la característica más influyente según el modelo, lo cual tiene sentido intuitivamente, ya que el uso de datos podría ser un buen indicador de la necesidad de un plan más robusto como el Ultra.
minutes: 22.98% - La duración de las llamadas también es significativa, lo que es razonable dado que podría correlacionarse con un uso más intensivo del plan.
calls: 21.87% - Similar a minutes, el número de llamadas también influye notablemente en la clasificación.
messages: 21.61% - Aunque también es importante, tiene un poco menos de peso que las llamadas y los minutos, lo que podría reflejar una menor variabilidad en el número de mensajes entre los usuarios de diferentes planes o una menor relevancia de los mensajes en la decisión del plan.